##### import numpy as np
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,f1_score,recall_score

In [2]:
train_dir = 'C:/Users/Jyoti Prakash Dhala/OneDrive/Desktop/brain_tumer_dataset/Training'
test_dir = 'C:/Users/Jyoti Prakash Dhala/OneDrive/Desktop/brain_tumer_dataset/Testing'

In [3]:
img_size = (224, 224)
batch_size = 16

In [4]:
datagen = ImageDataGenerator(rescale=1./255)

In [5]:

train_gen = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 5712 images belonging to 4 classes.


In [6]:
test_gen = datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 1311 images belonging to 4 classes.


In [7]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224,224,3))

In [8]:
model = Model(inputs=base_model.input, outputs=base_model.output)

In [9]:
from tqdm import tqdm  
def extract_features_batchwise(model, generator):
    features_list = []
    steps = len(generator)
    for i in tqdm(range(steps)):
        x, _ = generator[i]
        features = model.predict(x, verbose=0)
        features_list.append(features)
    features_array = np.vstack(features_list)
    return features_array

In [10]:
X_train = extract_features_batchwise(model, train_gen)
X_test = extract_features_batchwise(model, test_gen)

100%|██████████████████████████████████████████████████████████████████████████████████| 82/82 [01:28<00:00,  1.08s/it]


In [11]:
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

In [13]:
print(X_train.shape)
print(X_test.shape)

(5712, 51200)
(1311, 51200)


In [14]:
pca = PCA(n_components=500)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [15]:
y_train = train_gen.classes
y_test = test_gen.classes

In [16]:
svm = SVC(kernel='rbf', C=1, gamma='scale')
svm.fit(X_train_pca, y_train)
y_pred = svm.predict(X_test_pca)

In [17]:
print('accuracy:',accuracy_score(y_test, y_pred))
print('precision:',precision_score(y_test, y_pred, average='macro'))
print('recall:',recall_score(y_test, y_pred, average='macro'))
print('f1_score:',f1_score(y_test, y_pred, average='macro')) 

accuracy: 0.9366895499618612
precision: 0.9344499930739902
recall: 0.9319389978213507
f1_score: 0.9322675544649779


In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier

In [19]:
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train_pca, y_train)
y_pred_rf = rf.predict(X_test_pca)

In [20]:
print('accuracy:',accuracy_score(y_test, y_pred_rf))
print('precision:',precision_score(y_test, y_pred_rf, average='macro'))
print('recall:',recall_score(y_test, y_pred_rf, average='macro'))
print('f1_score:',f1_score(y_test, y_pred_rf, average='macro'))

accuracy: 0.9016018306636155
precision: 0.8950431470680233
recall: 0.8936274509803921
f1_score: 0.8925326824553882


In [21]:
lr = LogisticRegression(max_iter=1000, multi_class='multinomial')
lr.fit(X_train_pca, y_train)
y_pred_lr = lr.predict(X_test_pca)

D:\python\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [22]:
print('accuracy:',accuracy_score(y_test, y_pred_lr))
print('precision:',precision_score(y_test, y_pred_lr, average='macro'))
print('recall:',recall_score(y_test, y_pred_lr, average='macro'))
print('f1_score:',f1_score(y_test, y_pred_lr, average='macro'))

accuracy: 0.9260106788710908
precision: 0.920355269904553
recall: 0.9200526506899055
f1_score: 0.919960312822075


In [23]:
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', num_class=4, eval_metric='mlogloss')
xgb_clf.fit(X_train_pca, y_train)
y_pred_xgb = xgb_clf.predict(X_test_pca)

In [24]:
print('accuracy:',accuracy_score(y_test, y_pred_xgb))
print('precision:',precision_score(y_test, y_pred_xgb, average='macro'))
print('recall:',recall_score(y_test, y_pred_xgb, average='macro'))
print('f1_score:',f1_score(y_test, y_pred_xgb, average='macro'))

accuracy: 0.9107551487414187
precision: 0.9045803720735786
recall: 0.9033986928104575
f1_score: 0.90329879230067


In [25]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_pca, y_train)
y_pred_knn = knn.predict(X_test_pca)

In [26]:
print('accuracy:',accuracy_score(y_test, y_pred_knn))
print('precision:',precision_score(y_test, y_pred_knn, average='macro'))
print('recall:',recall_score(y_test, y_pred_knn, average='macro'))
print('f1_score:',f1_score(y_test, y_pred_knn, average='macro'))

accuracy: 0.9023646071700991
precision: 0.8961764095205055
recall: 0.8965196078431372
f1_score: 0.8952196545671025


In [27]:
import lightgbm as lgb
lgb_clf = lgb.LGBMClassifier(objective='multiclass', num_class=4)
lgb_clf.fit(X_train_pca, y_train)
y_pred_light = lgb_clf.predict(X_test_pca)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.044448 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 127500
[LightGBM] [Info] Number of data points in the train set: 5712, number of used features: 500
[LightGBM] [Info] Start training from score -1.464180
[LightGBM] [Info] Start training from score -1.450646
[LightGBM] [Info] Start training from score -1.275695
[LightGBM] [Info] Start training from score -1.366190


D:\python\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [28]:
print('accuracy:',accuracy_score(y_test, y_pred_light))
print('precision:',precision_score(y_test, y_pred_light, average='macro'))
print('recall:',recall_score(y_test, y_pred_light, average='macro'))
print('f1_score:',f1_score(y_test, y_pred_light, average='macro'))

accuracy: 0.916094584286804
precision: 0.9098860005756961
recall: 0.9091013071895424
f1_score: 0.90922265084153
